# Baseline SCA Measurements using Shunt Resistor on Target

The following script was used to capture the baseline measurements, first for T-Test, and then for CPA.

The firmware is also built in this notebook to keep a constant binary image for later comparison.

Note both sync & async captures leave the clock output enabled. This could be turned off as it increases noise a bit, but to leave the closest comparison between the two I've used the same binaries.

In [ ]:
import numpy as np
import chipwhisperer as cw
import os

In [ ]:
%run "functions.ipynb"

In [ ]:
data_dir = "d:/data_store"

## ChipWhisperer Configuration (Shared)

In [ ]:
# Set hardware settings
SCOPETYPE = 'OPENADC'
PLATFORM = 'CW308_SAM4S'
CRYPTO_TARGET='TINYAES128C' # 'TINYAES128C' or 'MBEDTLS'
SS_VER='SS_VER_2_1'

In [ ]:
# Connect to ChipWhisperer
%run "../../../Setup_Scripts/Setup_Generic.ipynb"

### Build Firmware Files & Cache for future use
You must pass extra defines to the makefile to build various firmwares:

```
C_EXTRA_DEFS="-DUSE_PLL -DUSE_EMBEDDED_CLOCK -DPLL_120MHZ"
```

Clock Source Options:
* `-DUSE_EXTERNAL_CLOCK`: Uses a 12 MHz clock input, must setup ChipWhisperer to generate that
* `-DUSE_EMBEDDED_CLOCK`: Uses a 12 MHz internal clock (not as stable as an XTAL)

General Options:
* `-DUSE_PLL`: Uses PLL, without this runs from external clock directly (normal ChipWhisperer HAL)
* `-DADD_PLL_JITTER1`: First strategy to add PLL jitter, sets `PMC_OCR` register before each AES encryption.

PLL Setup Options:
* `-DPLL_120MHZ`: PLL outputs 120 MHz to MCU core
* `-DPLL_15MHZ`: PLL outputs 15 MHz to MCU core
* `-DPLL_2MHZ`: PLL outputs 2 MHz to MCU core

Output Options:
* `-DPCLK_ENABLED`: Turns on PCLK output, which is always 1/2 of the MCU core frequency (e.g., 60/7.5/1MHz)
* `-PCLK_PIN_LOW`: If `-DPCLK_ENABLED` is NOT included the I/O pin is normally driven HIGH. With this define it will be driven LOW instead. Do not use this in combination with `-DPCLK_ENABLED` or you will have an invalid build.


In [ ]:
C_EXTRA_DEFS="-DUSE_PLL -DUSE_EMBEDDED_CLOCK -DPLL_120MHZ -DPCLK_ENABLED"

In [ ]:
%%bash -s "$PLATFORM" "$CRYPTO_TARGET" "$SS_VER" "$C_EXTRA_DEFS"
# compile firmware
cd ../colin-hacktest1/firmware/simpleserial-aes-spitest
make PLATFORM=$1 CRYPTO_TARGET=$2 SS_VER=$3 C_EXTRA_DEFS="$4" -j
cp simpleserial-aes-CW308_SAM4S.hex  ../../../sam4s-comparisons/firmwares/fw-120mhz-clkout.hex

In [ ]:
C_EXTRA_DEFS="-DUSE_PLL -DUSE_EMBEDDED_CLOCK -DPLL_15MHZ -DPCLK_ENABLED"

In [ ]:
%%bash -s "$PLATFORM" "$CRYPTO_TARGET" "$SS_VER" "$C_EXTRA_DEFS"
# compile firmware
cd ../colin-hacktest1/firmware/simpleserial-aes-spitest
make PLATFORM=$1 CRYPTO_TARGET=$2 SS_VER=$3 C_EXTRA_DEFS="$4" -j
cp simpleserial-aes-CW308_SAM4S.hex  ../../../sam4s-comparisons/firmwares/fw-15mhz-clkout.hex

In [ ]:
C_EXTRA_DEFS="-DUSE_PLL -DUSE_EMBEDDED_CLOCK -DPLL_2MHZ -DPCLK_ENABLED"

In [ ]:
%%bash -s "$PLATFORM" "$CRYPTO_TARGET" "$SS_VER" "$C_EXTRA_DEFS"
# compile firmware
cd ../colin-hacktest1/firmware/simpleserial-aes-spitest
make PLATFORM=$1 CRYPTO_TARGET=$2 SS_VER=$3 C_EXTRA_DEFS="$4" -j
cp simpleserial-aes-CW308_SAM4S.hex  ../../../sam4s-comparisons/firmwares/fw-2mhz-clkout.hex

In [ ]:
C_EXTRA_DEFS="-DUSE_PLL -DUSE_EMBEDDED_CLOCK -DPLL_120MHZ"

In [ ]:
%%bash -s "$PLATFORM" "$CRYPTO_TARGET" "$SS_VER" "$C_EXTRA_DEFS"
# compile firmware
cd ../colin-hacktest1/firmware/simpleserial-aes-spitest
make PLATFORM=$1 CRYPTO_TARGET=$2 SS_VER=$3 C_EXTRA_DEFS="$4" -j
cp simpleserial-aes-CW308_SAM4S.hex  ../../../sam4s-comparisons/firmwares/fw-120mhz.hex

In [ ]:
C_EXTRA_DEFS="-DUSE_PLL -DUSE_EMBEDDED_CLOCK -DPLL_15MHZ"

In [ ]:
%%bash -s "$PLATFORM" "$CRYPTO_TARGET" "$SS_VER" "$C_EXTRA_DEFS"
# compile firmware
cd ../colin-hacktest1/firmware/simpleserial-aes-spitest
make PLATFORM=$1 CRYPTO_TARGET=$2 SS_VER=$3 C_EXTRA_DEFS="$4" -j
cp simpleserial-aes-CW308_SAM4S.hex  ../../../sam4s-comparisons/firmwares/fw-15mhz.hex

In [ ]:
C_EXTRA_DEFS="-DUSE_PLL -DUSE_EMBEDDED_CLOCK -DPLL_2MHZ"

In [ ]:
%%bash -s "$PLATFORM" "$CRYPTO_TARGET" "$SS_VER" "$C_EXTRA_DEFS"
# compile firmware
cd ../colin-hacktest1/firmware/simpleserial-aes-spitest
make PLATFORM=$1 CRYPTO_TARGET=$2 SS_VER=$3 C_EXTRA_DEFS="$4" -j
cp simpleserial-aes-CW308_SAM4S.hex  ../../../sam4s-comparisons/firmwares/fw-2mhz.hex

Set default settings for rest of captures (will be overridden for some captures):
* No clock driven to target
* Gain mode works for most captures (could be adjusted)
* 50K samples (could be adjusted)

In [ ]:
scope.io.hs2 = None
scope.gain.mode = "low"
scope.gain.gain = 45

In [ ]:
scope.adc.samples = 50000

Make a live plot to monitor captures

In [ ]:
splot = cw.StreamPlot()
splot.plot()

## T-Test Captures

In [ ]:
# program firmware onto target
cw.program_target(scope, prog, "firmwares/fw-120mhz-clkout.hex")
scope.io.target_pwr = False
time.sleep(0.25)
scope.io.target_pwr = True

In [ ]:
#Baud is lower so it works from internal oscillator which isn't as precise
target.baud = 38400

In [ ]:
scope.io.hs2 = None

In [ ]:
#syncronous - 120 MS/s ADC setting
def clocksetup(syncstr, fmcu, msps):
    if syncstr == "sync":
        scope.clock.clkgen_src = "extclk"
        scope.io.hs2 = None

        if msps == 120:
            scope.clock.pll.set_outfreqs(fmcu*1E6/2, 60E6, 2, True)
        elif msps == 60:
            scope.clock.pll.set_outfreqs(fmcu*1E6/2, 60E6, 1, True)
        else:
            raise ValueError
    elif syncstr == "async":
        scope.clock.clkgen_src = "internal"
        scope.clock.clkgen_freq = 60E6
        if msps == 120:
            scope.clock.adc_mul = 2
        elif msps == 60:
            scope.clock.adc_mul = 1
        else:
            raise ValueError
    else:
        raise ValueError

In [ ]:
def basetcapture(syncstr, fmcu, msps):
    N = 10000
    clocksetup(syncstr, fmcu, msps)
    group1, group2 = capture_ttest(N, picoscope=False, splot=splot)
    nptsave("baseline{:d}mhz_{:s}_{:d}msps_ttest_10k".format(fmcu, syncstr, msps), group1, group2)

In [ ]:
basetcapture("sync", 120, 120)

In [ ]:
basetcapture("sync", 120, 60)

In [ ]:
basetcapture("async", 120, 120)

In [ ]:
basetcapture("async", 120, 60)

In [ ]:
# program firmware onto target
cw.program_target(scope, prog, "firmwares/fw-15mhz-clkout.hex")
scope.io.target_pwr = False
time.sleep(0.25)
scope.io.target_pwr = True

In [ ]:
basetcapture("sync", 15, 120)

In [ ]:
basetcapture("sync", 15, 60)

In [ ]:
basetcapture("async", 15, 120)

In [ ]:
basetcapture("async", 15, 60)

In [ ]:
# program firmware onto target
cw.program_target(scope, prog, "firmwares/fw-2mhz-clkout.hex")
scope.io.target_pwr = False
time.sleep(0.25)
scope.io.target_pwr = True

The 2MHz firmware has a different baud as it can't calculate the same 38400 one

In [ ]:
target.baud = 62750

In [ ]:
scope.gain.gain = 20
scope.gain.mode = "high"

Sync capture isn't possible. With a 2MHz CPU, the output clock is 1 MHz, which falls below the PLL input range to be able to calculate up to 120 or 60 MHz. So we just ignore these.

In [ ]:
#basetcapture("sync", 2, 120)

In [ ]:
#basetcapture("sync", 2, 60)

In [ ]:
scope.adc.samples = 100000
scope.adc.offset = 100000

In [ ]:
basetcapture("async", 2, 120)

In [ ]:
scope.adc.samples = 100000
scope.adc.offset = 50000

In [ ]:
basetcapture("async", 2, 60)

## SCA Captures

In [ ]:
# program firmware onto target
cw.program_target(scope, prog, "firmwares/fw-120mhz-clkout.hex")
scope.io.target_pwr = False
time.sleep(0.25)
scope.io.target_pwr = True

In [ ]:
#Baud is lower so it works from internal oscillator which isn't as precise
target.baud = 38400

In [ ]:
scope.io.hs2 = None

In [ ]:
splot = cw.StreamPlot()
splot.plot()

In [ ]:
#Don't need as many samples for these
scope.adc.offset = 0
scope.adc.samples = 20000

In [ ]:
clocksetup("sync", 120, 120)

In [ ]:
waves, textins, textouts, keys = capture_cpa(1000, splot=splot)

In [ ]:
save_ets(waves, textins, textouts, keys, "./data_store/baseline120mhz_sync_120msps_cpa.ets", overwrite=False)

In [ ]:
clocksetup("sync", 120, 60)

In [ ]:
waves, textins, textouts, keys = capture_cpa(4000, splot=splot)

In [ ]:
save_ets(waves, textins, textouts, keys, "./data_store/baseline120mhz_sync_60msps_cpa.ets", overwrite=False)

In [ ]:
scope.adc.offset = 0
scope.adc.samples = 6000
clocksetup("async", 120, 120)

In [ ]:
waves, textins, textouts, keys = capture_cpa(80000, splot=splot)

In [ ]:
save_ets(waves, textins, textouts, keys, "./data_store/baseline120mhz_async_120msps_cpa.ets", overwrite=False)

In [ ]:
clocksetup("async", 120, 60)

In [ ]:
waves, textins, textouts, keys = capture_cpa(80000, splot=splot)

In [ ]:
save_ets(waves, textins, textouts, keys, "./data_store/baseline120mhz_async_60msps_cpa.ets", overwrite=False)

In [ ]:
# program firmware onto target
cw.program_target(scope, prog, "firmwares/fw-15mhz-clkout.hex")
scope.io.target_pwr = False
time.sleep(0.25)
scope.io.target_pwr = True

In [ ]:
scope.adc.offset = 11000
scope.adc.samples = 10000

In [ ]:
clocksetup("sync", 15, 120)

In [ ]:
waves, textins, textouts, keys = capture_cpa(5000, splot=splot)

In [ ]:
save_ets(waves, textins, textouts, keys, "./data_store/baseline15mhz_sync_120msps_cpa.ets", overwrite=False)

In [ ]:
scope.adc.offset = int(11000/2)
scope.adc.samples = int(10000/2)

In [ ]:
clocksetup("sync", 15, 60)

In [ ]:
waves, textins, textouts, keys = capture_cpa(5000, splot=splot)

In [ ]:
save_ets(waves, textins, textouts, keys, "./data_store/baseline15mhz_sync_60msps_cpa.ets", overwrite=False)

In [ ]:
scope.adc.offset = 11000
scope.adc.samples = 10000

In [ ]:
clocksetup("async", 15, 120)

In [ ]:
waves, textins, textouts, keys = capture_cpa(40000, splot=splot)

In [ ]:
save_ets(waves, textins, textouts, keys, "./data_store/baseline15mhz_async_120msps_cpa.ets", overwrite=True)

In [ ]:
scope.adc.offset = int(11000/2)
scope.adc.samples = int(10000/2)

In [ ]:
clocksetup("async", 15, 60)

In [ ]:
waves, textins, textouts, keys = capture_cpa(50000, splot=splot)

In [ ]:
save_ets(waves, textins, textouts, keys, "./data_store/baseline15mhz_async_60msps_cpa.ets", overwrite=False)

In [ ]:
# program firmware onto target
cw.program_target(scope, prog, "firmwares/fw-2mhz-clkout.hex")
scope.io.target_pwr = False
time.sleep(0.25)
scope.io.target_pwr = True

In [ ]:
target.baud = 62750

In [ ]:
scope.gain.gain = 20
scope.gain.mode = "high"

In [ ]:
scope.adc.samples = 130000
scope.adc.offset = 50000

In [ ]:
clocksetup("async", 2, 120)

In [ ]:
waves, textins, textouts, keys = capture_cpa(5000, splot=splot)

In [ ]:
save_ets(waves, textins, textouts, keys, "./data_store/baseline2mhz_async_120msps_cpa.ets", overwrite=True)

In [ ]:
scope.adc.samples = 130000
scope.adc.offset = 50000

In [ ]:
clocksetup("async", 2, 60)

In [ ]:
waves, textins, textouts, keys = capture_cpa(20000, splot=None)

In [ ]:
save_ets(waves, textins, textouts, keys, "d:/data_store/baseline2mhz_async_60msps_cpa.ets", overwrite=True)

You can use the following to get a rough idea of length of AES rounds, or plot the waveforms to ensure things look right:

In [ ]:
scope.adc.trig_count / 10

In [ ]:
plt.plot(waves[0])